In [1]:
import numpy as np
import pandas as pd
import scipy
from typing import TypeVar,Mapping, Set, Generic, Sequence, Callable

### Epsilon-Greedy Policy Improvement Theorem
Definition of $\epsilon$-greedy: $\pi(a|s) = \epsilon/m + 1 -\epsilon$ for $a = \arg \max_a Q(s,a)$ and $\pi(a|s) = \epsilon/m$ otherwise.
Proof: If $\pi'$ is the $\epsilon$-greedy improvement from $Q_{\pi}(s,a)$.
$$
\begin{split}
Q_{\pi}(s,\pi'(s)) &= \sum_{a \in A} \pi'(a|s) Q_{\pi}(s,a)\\
&= \epsilon/m \sum_{a \in A} Q_{\pi}(s,a) + (1 -\epsilon) \max_{a \in A} Q_{\pi}(s,a)\\
& \geq \epsilon/m \sum_{a \in A} Q_{\pi}(s,a) + (1 -\epsilon) \sum_{a \in A} \frac{\pi(a|s) - \epsilon/m}{1 -\epsilon}Q_{\pi}(s,a)\\
& \geq \sum_{a \in A}\pi(a|s) Q_{\pi}(s,a)\\
&= V_{\pi}(s)
\end{split}
$$
Thus $V_{\pi'}(s) \geq V_{\pi}(s)$.

### Greedy in the Limit with Infinite Exploration
- All state-action pairs are explored infinitely many times (ensure exploration)
$$\lim_{k \rightarrow \infty} N_k (s,a) = \infty$$
- The policy converges on a greedy policy (ensure explotation)
$$\lim_{k \rightarrow \infty} \pi_k (a|s) = \mathbb{1}(a = \arg \max_{a' \in A} Q_k (s,a'))$$

###  Sarsa
$$Q(S,A) \leftarrow Q(S,A) + \alpha (R + \gamma Q(S',A') -Q(S,A))$$

In [3]:
# On-policy Sarsa
# Step 1: generate epsilon greedy policy
def epsilon_greedy(Q: np.ndarray, S:int, eps:float) -> np.ndarray:
    pol = eps*np.ones(Q.shape[1])/Q.shape[1]
    A_ast = np.argmax(Q[S])
    pol[A_ast] += 1-eps
    return pol

# Step 2: prepare interface
class MDPforRL_TB():
    
    # note that state and actions are defined as int in this part 
    # extension 1: can create a dict of state2idx outside of the class
    # extension 2: can change input simulator to s RV generation function (vs. pre-defined np)
    def __init__(self, state_action_tab: dict, # get available actions from each state
                 state_action_simulator: np.ndarray, # probability of [s_cur,a,s_next]
                 reward_simulator: np.ndarray, # reward if [s_cur,a]
                 gamma: float) -> None:
        super(MDPforRL_TB, self).__init__()
        self.state = state_action_tab.keys()
        self.action = state_action_tab[0]
        self.state_action_tab = state_action_tab
        self.state_action_simulator = state_action_simulator
        self. reward_simulator =  reward_simulator
        self.gamma = gamma
    
    def get_state_action_simulator():
        return self.state_action_simulator
    
    def gen_init_state(self): # uniform start
        init_state = np.random.choice(len(self.state_action_simulator), 1)[0]
        return init_state
    
    def gen_next_state_reward(self,S,A):
        u = np.random.uniform(0,1)
        cdf = np.cumsum(self.state_action_simulator[S,A,:])
        next_state = np.where(cdf > u)[0][0]
        step_reward = self.reward_simulator[S,A]
        return next_state, step_reward
    
    def get_avail_actions(self, S):
        return self.state_action_tab[S]
    
    def get_state(self):
        return list(self.state)
    
    def n_S(self):
        return len(list(self.state))
    
    def n_A(self):
        return len(list(self.action))
    
class tab_RL_interface():

    def __init__(self, mdp: MDPforRL_TB):
        super(tab_RL_interface).__init__()
        self.mdp = mdp
    
    # Generate initial step
    def init_state_gen(self) -> tuple:
        return mdp.gen_init_state()
    
    # Generate next step
    def next_state_gen(self, cur_state: int, cur_act: int) -> tuple:
        return mdp.gen_next_state_reward(cur_state,cur_act)
    
    # Get available actions
    def get_avail_actions(self, cur_state):
        return mdp.get_avail_actions(cur_state)
    
    # Get states
    def get_states(self):
        return list(self.mdp.get_state())
    
    def n_S(self):
        return self.mdp.n_S()
    
    def n_A(self):
        return self.mdp.n_A()

# Step 3: Sarsa
def sarsa(tb_rl:tab_RL_interface, num_episode: int, len_episode: int, 
          alpha: float, gamma: float, epsilon_greedy: Callable[[int], int], eps: float) -> dict:
    
    # Initiate q_func table
    q_func = np.zeros((tb_rl.n_S(), tb_rl.n_A()))
    i = 0
    while i < num_episode:
        j = 0
        s_cur = tb_rl.init_state_gen()
        act_cur = epsilon_greedy(q_func, s_cur, eps)
        while j < len_episode:
            s_next, reward = tb_rl.next_state_gen(s_cur,act)
            act_next = epsilon_greedy(q_func, s_next, eps)
            q_func[s_cur, act_cur] += alpha*(reward + gamma*q_func[s_next, act_next] - q_func[s_cur, act_cur])
            s_cur = s_next
            act_cur = act_next
            j += 1
        i += 1
    
    return q_func

- Forward view Sarsa($\lambda$):
$$Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha (q_t^{\lambda} -Q(S_t,A_t))$$
$$q_t^{\lambda} = (1-\lambda) \sum_{n=1}^{\infty} \lambda^{n-1} q_t^{(n)}$$
$$q_t^{(n)} = R_{t+1} + \gamma R_{t+2}+...+\gamma^n Q(S_{t+n})$$
- Backward view Sarsa($\lambda$):
$$\delta_t = R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)$$
$$\mathbb{E}_t(s,a) = \gamma \lambda \mathbb{E}_{t-1}(s,a) + \mathbb{1}(S_t = s, A_t = a)$$
$$Q(s, a) = Q(s,a) + \alpha \delta_t \mathbb{E}_t(s, a) | \mathbb{E}_0(s, a) = 0$$

In [4]:
# Step 1 & 2: generate epsilon greedy policy and prepare interface as above
# Step 3: Backward Sarsa
def Sarsa_backward(tb_rl:tab_RL_interface, num_episode: int, len_episode: int,
               alpha: float, gamma: float, _lambda: float, eps: float,
               epsilon_greedy: Callable[[int], int]) -> dict:

    # Initiate q_func table
    q_func = np.zeros((tb_rl.n_S(), tb_rl.n_A()))
    e_t = np.zeros((tb_rl.n_S(), tb_rl.n_A()))
    
    i = 0
    while i < num_episode:
        j = 0
        s_cur = tb_rl.init_state_gen()
        a_cur = epsilon_greedy(q_func, s_cur, eps)
        while j < len_episode:
            s_next, reward = tb_rl.next_state_gen(s_cur,a_cur)
            a_next = epsilon_greedy(q_func, s_next, eps)
            e_t *= _lambda * gamma
            e_t[s_cur, a_cur] += 1.0
            error = reward + gamma * q_func[s_next, a_next] - q_func[s_cur, a_cur]
            q_func[s_cur, a_cur] += alpha*(reward + gamma*q_func[s_next, a_next] - q_func[s_cur, a_cur])
            s_cur = s_next
            a_cur = a_next
            q_func += alpha * error * e_t
            j += 1
        i += 1
    return q_func

### Q-Learning
$$Q(S,A) \leftarrow Q(S,A) + \alpha (R + \gamma \max_{A'} Q(S',A') -Q(S,A))$$

In [5]:
# Step 1 & 2: generate epsilon greedy policy and prepare interface as above
# Step 3: Q-learning
def greedy(Q: np.ndarray, S:int) -> int:
    A_ast = np.argmax(Q[S])
    return A_ast

def q_learning(tb_rl:tab_RL_interface, num_episode: int, len_episode: int, 
               alpha: float, gamma: float, epsilon_greedy: Callable[[int], int], greedy: Callable[[int], int], 
               eps: float) -> dict:
    
    # Initiate q_func table
    q_func = np.zeros((tb_rl.n_S(), tb_rl.n_A()))
    i = 0
    while i < num_episode:
        j = 0
        s_cur = tb_rl.init_state_gen()
        while j < len_episode:
            act_cur = epsilon_greedy(q_func, s_cur, eps)
            s_next, reward = tb_rl.next_state_gen(s_cur,act)
            act_next = greedy(q_func, s_next)
            q_func[s_cur, act_cur] += alpha*(reward + gamma*q_func[s_next, act_next] - q_func[s_cur, act_cur])
            s_cur = s_next
            j += 1
        i += 1
    
    return q_func